# Broadband evolution in New Jersey

This is an example on how to use the Microsoft Open Data for Society Broadband files. It was designed to work with minimal imports.

In [ ]:
# We chose to use Plotly Express because it's free to use, multiplatform and easy to set up. 
# GeoPandas and others give more flexibility but may be harder to set up and use
import json
import pandas as pd
import plotly.express as px

## Collect the data

This notebook was created for a demo for Rutgers University. Therefore, we filtered the data to New Jersey (where Rutgers is) in order to make the notebook lighter.

In [ ]:
# Load the broadband data for 2019 and filter it to New Jersey
df_2019 = pd.read_csv("data/broadband_data_2019November.csv")
df_2019 = df_2019.query("ST=='NJ'").reset_index()
df_2019 = df_2019[["ST", "COUNTY ID", "COUNTY NAME", " BROADBAND AVAILABILITY PER FCC ", " BROADBAND USAGE "]] # mind the spaces in the column name
df_2019.columns = ['state', 'county', 'county_name', 'broadband_availability_2019', 'broadband_usage_2019']

# Numbers are stored as text, convert them to numbers
df_2019["broadband_availability_2019"] = df_2019["broadband_availability_2019"].astype(float)
df_2019["broadband_usage_2019"] = df_2019["broadband_usage_2019"].astype(float)

df_2019

In [ ]:
# Load the broadband data for 2020 and filter it to New Jersey
df_2020 = pd.read_csv("data/broadband_data_2020October.csv")
df_2020 = df_2020.query("ST=='NJ'").reset_index()
df_2020 = df_2020[["ST", "COUNTY ID", "COUNTY NAME", " BROADBAND AVAILABILITY PER FCC ", " BROADBAND USAGE "]] # mind the spaces in the column name
df_2020.columns = ['state', 'county', 'county_name', 'broadband_availability_2020', 'broadband_usage_2020']

# Numbers are stored as text, convert them to numbers
df_2020["broadband_availability_2020"] = df_2020["broadband_availability_2020"].astype(float)
df_2020["broadband_usage_2020"] = df_2020["broadband_usage_2020"].astype(float)

df_2020

## Combine data from 2019 and 2020

We're going to combine the data from 2019 and 2020 so that we can see the change between these two years.


In [ ]:
df_combined = pd.merge(df_2019, df_2020, on=["state", "county", "county_name"])
df_combined

## Read the shapefile

When using geographical information, we usually need "shapefiles" that have data about the boundaries of geographical entities. This shapefile has the boundaries of counties. Shapefiles tend to be quite big.

In [ ]:
with open("shapefiles/geojson-counties-fips.json") as f:
    counties = json.load(f)

# Explore the data using maps

One interesting thing to observe with this data is that you can reach very different conclusions when looking at broadband **availability** (data provided by the FCC) and broadband **usage** (data provided by this dataset). The FCC defines availability as whether somebody living in that regions *could* have high-speed internet. This open dataset measures their average *actual* download speed to see if people are actually getting the advertised speeds.

There are many possible reasons for potential discrepancies. One reason is when high-speed internet is available for some population in the county (e.g., in the main city), but high investments are necessary to bring the high-speed to rural dwellings.

## Availability

In [ ]:
df_combined["availability_delta"] = df_combined["broadband_availability_2020"] - df_combined["broadband_usage_2020"]

fig = px.choropleth(df_combined, geojson=counties, locations='county', color='broadband_availability_2020',
                           color_continuous_scale="armyrose_r",
                           range_color=(0, 1),
                           scope="usa",
                           labels={'broadband_availability_2020':'availability'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.add_scattergeo(
                   locations = df_combined['county'],
                   text = round(100*df_combined['broadband_availability_2020'], 1),
                   geojson=counties,
                   mode = 'text',
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Usage

Microsoft compiles this data, which represents *actual* obtained speeds. As you will see, the story is quite different.

In [ ]:
fig = px.choropleth(df_combined, geojson=counties, locations='county', color='broadband_usage_2020',
                           color_continuous_scale="armyrose_r",
                           range_color=(0, 1),
                           scope="usa",
                           labels={'broadband_availability_2020':'availability'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.add_scattergeo(
                   locations = df_combined['county'],
                   text = round(100*df_combined['broadband_usage_2020'], 1),
                   geojson=counties,
                   mode = 'text',
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Availability to usage delta

This is the difference between the availability and the usage. The picture will be very similar to the usage.

In [ ]:
# This next line calculates the difference between availability and usage in 2020
df_combined["availability_delta"] = df_combined["broadband_availability_2020"] - df_combined["broadband_usage_2020"]

fig = px.choropleth(df_combined, geojson=counties, locations='county', color='availability_delta',
                           color_continuous_scale="armyrose",
                           range_color=(0, 1),
                           scope="usa",
                           labels={'broadband_availability_2020':'availability'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.add_scattergeo(
                   locations = df_combined['county'],
                   text = round(100*df_combined['availability_delta'], 1),
                   geojson=counties,
                   mode = 'text',
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Usage delta between 2019 and 2020

In [ ]:
# This next line calculates the difference between availability and usage in 2019 and 2020
df_combined["usage_delta"] = df_combined["broadband_usage_2020"] - df_combined["broadband_usage_2019"]

fig = px.choropleth(df_combined, geojson=counties, locations='county', color='usage_delta',
                           color_continuous_scale="armyrose_r",
                           range_color=(-0.2, 0.2),
                           scope="usa",
                           labels={'broadband_availability_2020':'availability'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.add_scattergeo(
                   locations = df_combined['county'],
                   text = round(100*df_combined['usage_delta'], 1),
                   geojson=counties,
                   mode = 'text',
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


# Looking into demographics

I've downloaded demographic data from [IPUMS](https://www.ipums.org/) to take a deeper look into determinants of low broadband use.

This is not a scientific analysis to determine policy, it's just a directional example to show how to use this data. Some examples of shortcomings: the data was not weighted and just about 5,000 samples were collected from IPUMS. The summarized data only has 14 observations, which is definitely not enough for statistical tests, but might provide some interesting directions.

In [ ]:
df_demographics = pd.read_csv("data/demographics_clean.csv")
df_demographics

In [ ]:



mod = smf.ols(formula="broadband_usage_2020 ~ college_4yr_or_more + household_income + employed + hispanic", data=df_regression)

# Fit the model
res = mod.fit()

res.summary()


# Zip Code level data

In 2020, Microsoft also started to make zip code level data available. This allows a lot more granularity and nuance in understanding broadband usage.

To start, we need to load the shapefile for zip codes. Note: this file is very big.

In [ ]:
with open("shapefiles/nj_new_jersey_zip_codes_geo.min.json") as f:
    zipcodes = json.load(f)

In [ ]:
bb_zip_df = pd.read_csv("data/broadband_data_zipcode.csv")

# Filter on New Jersey
bb_zip_df = bb_zip_df.query("ST=='NJ'")[["POSTAL CODE", "BROADBAND USAGE"]]

# Make sure that the zipcode has 5 digits with a leading zero
bb_zip_df["POSTAL CODE"] = bb_zip_df["POSTAL CODE"].apply(str)
bb_zip_df["POSTAL CODE"] = bb_zip_df["POSTAL CODE"].str.pad(5, "left", "0")


In [ ]:
bb_zip_df

In [ ]:
fig = px.choropleth(bb_zip_df, geojson=zipcodes, locations='POSTAL CODE', color='BROADBAND USAGE',
                           color_continuous_scale="armyrose_r",
                           range_color=(0, 1),
                           featureidkey="properties.ZCTA5CE10",
                           scope="usa",
                           labels={'BROADBAND USAGE':'usage'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Where to go from here?

This is just an initial tour of the broadband data. You can join this data with county-level data or even zip-code level data to try to answer lots of interesting questions:

- Does income correlate with higher broadband use? (expected yes)
- Do areas with higher broadband use have better educational outcomes?
- Is there a correlation between increased density and broadband usage?
- Is there a correlation between infrastructure investments and broadband usage?